In [ ]:
!pip uninstall keras -y
!pip install git+https://github.com/qubvel/segmentation_models
!git clone https://github.com/SlinkoIgor/ImageDataAugmentor.git

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)




#add your input file path here
images_radiopedia = np.load('images_radiopedia.npy').astype(np.float32)
masks_radiopedia = np.load('masks_radiopedia.npy').astype(np.int8)
images_medseg = np.load('images_medseg.npy').astype(np.float32)
masks_medseg = np.load('masks_medseg.npy').astype(np.int8)

test_images_medseg = np.load(os.path.join(prefix, 'test_images_medseg.npy')).astype(np.float32)
import matplotlib.pyplot as plt
import numpy as np


def visualize(image_batch, mask_batch=None, pred_batch=None, num_samples=8):
    num_classes = mask_batch.shape[-1] if mask_batch is not None else 0
    fix, ax = plt.subplots(num_classes + 1, num_samples, figsize=(num_samples * 2, (num_classes + 1) * 2))

    for i in range(num_samples):
        ax_image = ax[0, i] if num_classes > 0 else ax[i]
        ax_image.imshow(image_batch[i,:,:,0], cmap='Greys')
        ax_image.set_xticks([]) 
        ax_image.set_yticks([])
        
        if mask_batch is not None:
            for j in range(num_classes):
                if pred_batch is None:
                    mask_to_show = mask_batch[i,:,:,j]
                else:
                    mask_to_show = np.zeros(shape=(*mask_batch.shape[1:-1], 3)) 
                    mask_to_show[..., 0] = pred_batch[i,:,:,j] > 0.5
                    mask_to_show[..., 1] = mask_batch[i,:,:,j]
                ax[j + 1, i].imshow(mask_to_show, vmin=0, vmax=1)
                ax[j + 1, i].set_xticks([]) 
                ax[j + 1, i].set_yticks([]) 

    plt.tight_layout()
    plt.show()
Images from radiopedia are full CT volumes:
Class 0 is "ground glass"
Class 1 is "consolidations"
Class 2 is "lungs other" – it doesn't mean that it is healthy lungs (you don't need to predict this class)
Class 3 is "background" – not lungs (you don't need to predict this class)

visualize(images_radiopedia[30:], masks_radiopedia[30:])
Images from medseg are individual axial slices:
Classes are same as in radiopedia part

visualize(images_medseg, masks_medseg)
Test images from medseg are individual axial slices:
You should make predictions for class 0 "ground glass" and class 1 "consolidation"

visualize(test_images_medseg)
def plot_hists(images1, images2=None):
    plt.hist(images1.ravel(), bins=100, density=True, color='b', alpha=1 if images2 is None else 0.5)
    if images2 is not None:
        plt.hist(images2.ravel(), bins=100, density=True, alpha=0.5, color='orange')
    plt.show();
Plot images hists:
HU (Hounsfield scale) of radiopedia data (blue) vs medseg data (orange):

plot_hists(images_radiopedia, images_medseg)
HU (Hounsfield scale) of test medseg data (blue) vs medseg data (orange):

plot_hists(test_images_medseg, images_medseg)
Preprocess images:
def preprocess_images(images_arr, mean_std=None):
    images_arr[images_arr > 500] = 500
    images_arr[images_arr < -1500] = -1500
    min_perc, max_perc = np.percentile(images_arr, 5), np.percentile(images_arr, 95)
    images_arr_valid = images_arr[(images_arr > min_perc) & (images_arr < max_perc)]
    mean, std = (images_arr_valid.mean(), images_arr_valid.std()) if mean_std is None else mean_std
    images_arr = (images_arr - mean) / std
    print(f'mean {mean}, std {std}')
    return images_arr, (mean, std)

images_radiopedia, mean_std = preprocess_images(images_radiopedia)
images_medseg, _ = preprocess_images(images_medseg, mean_std)
test_images_medseg, _ = preprocess_images(test_images_medseg, mean_std)
Normalized values of radiopedia data (blue) vs medseg data (orange):

plot_hists(images_radiopedia, images_medseg)
Normalized values of test medseg data (blue) vs medseg data (orange):

plot_hists(test_images_medseg, images_medseg)
Split train / val:
val_indexes, train_indexes = list(range(24)), list(range(24, 100))

train_images = np.concatenate((images_medseg[train_indexes], images_radiopedia))
train_masks = np.concatenate((masks_medseg[train_indexes], masks_radiopedia))
val_images = images_medseg[val_indexes]
val_masks = masks_medseg[val_indexes]

batch_size = len(val_masks)

del images_radiopedia
del masks_radiopedia
del images_medseg
del masks_medseg
Data generator and augmentations:
import tensorflow

import albumentations
import cv2

#for PSPNet
SOURCE_SIZE = 510
TARGET_SIZE = 384

train_augs = albumentations.Compose([
    albumentations.Rotate(limit=360, p=0.9, border_mode=cv2.BORDER_REPLICATE),
    albumentations.RandomSizedCrop((int(SOURCE_SIZE * 0.75), SOURCE_SIZE), 
                                   TARGET_SIZE, 
                                   TARGET_SIZE, 
                                   interpolation=cv2.INTER_NEAREST),
    albumentations.HorizontalFlip(p=0.5),

])

val_augs = albumentations.Compose([
    albumentations.Resize(TARGET_SIZE, TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
])
class Dataset:   
    def __init__(
            self, 
            images, 
            masks,
            augmentations=None
    ):
        self.images = images
        self.masks = masks
        self.augmentations = augmentations
    
    def __getitem__(self, i):
        image = self.images[i]
        mask = self.masks[i]
        
        if self.augmentations:
            sample = self.augmentations(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        return image, mask
        
    def __len__(self):
        return len(self.images)
    
    
class Dataloder(tensorflow.keras.utils.Sequence):
    """Load data from dataset and form batches
    
    Args:
        dataset: instance of Dataset class for image loading and preprocessing.
        batch_size: Integet number of images in batch.
        shuffle: Boolean, if `True` shuffle image indexes each epoch.
    """
    
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        # collect batch data
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        images = []
        masks = []
        for j in range(start, stop):
            image, mask = self.dataset[self.indexes[j]]
            images.append(image)
            masks.append(mask)
        
        images = np.stack(images, axis=0)
        masks = np.stack(masks, axis=0).astype(np.float32)
        
        return (images, masks)
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        """Callback function to shuffle indexes each epoch"""
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
            
train_dataset = Dataset(train_images, train_masks, train_augs)
val_dataset = Dataset(val_images, val_masks, val_augs)

train_dataloader = Dataloder(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = Dataloder(val_dataset, batch_size=batch_size, shuffle=False)
assert train_dataloader[0][0].shape == (batch_size, TARGET_SIZE, TARGET_SIZE, 1)
assert train_dataloader[0][1].shape == (batch_size, TARGET_SIZE, TARGET_SIZE, 4)
visualize(*next(iter(train_dataloader)))
visualize(*next(iter(val_dataloader)))
Metrics:
def fscore_glass(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 0:1], 
                               y_pred[..., 0:1])
    
def fscore_consolidation(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 1:2], 
                               y_pred[..., 1:2])

def fscore_lungs_other(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., 2:3], 
                               y_pred[..., 2:3])

def fscore_glass_and_consolidation(y_true, y_pred):
    return sm.metrics.f1_score(y_true[..., :2], 
                               y_pred[..., :2])
from segmentation_models import PSPNet
import segmentation_models as sm

from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
np.random.seed(0)

base_model = PSPNet(backbone_name='efficientnetb0',
                  encoder_weights='imagenet',
                  classes=4, 
                  activation='softmax')

model = Sequential([Input(shape=(TARGET_SIZE, TARGET_SIZE, 1)),
                    Conv2D(3, (1, 1)),  # map N channels data to 3 channels
                    base_model])
    
model.compile(Adam(learning_rate=0.001, amsgrad=True),
              loss=sm.losses.categorical_crossentropy,
              metrics=[fscore_glass, fscore_consolidation, fscore_lungs_other, fscore_glass_and_consolidation])

checkpoint_callback = ModelCheckpoint('best_model',
                                      monitor='fscore_glass_and_consolidation',
                                      mode='max',
                                      save_best_only=True)

model.fit(
    train_dataloader,
    steps_per_epoch=len(train_dataloader) * 6,
    epochs=10,
    validation_data=val_dataloader,
    validation_steps=len(val_dataloader),
    callbacks=[checkpoint_callback],
    workers=4)
Load best model and visualize predicions on val:
del train_images
del train_masks
model = tensorflow.keras.models.load_model('best_model/',
                                           compile=False,
                                           custom_objects={
                                                'categorical_crossentropy': sm.losses.categorical_crossentropy,
                                                'fscore_consolidation': fscore_consolidation, 
                                                'fscore_glass': fscore_glass, 
                                                'fscore_lungs_other': fscore_lungs_other,
                                                'fscore_glass_and_consolidation': fscore_glass_and_consolidation})

model.compile(Adam(learning_rate=0.001, amsgrad=True),
              loss=sm.losses.jaccard_loss)
input = val_dataloader[0]
image_batch, mask_batch = input

preds = model.predict_on_batch(image_batch)
visualize(image_batch, mask_batch, pred_batch=preds)

# yellow is TP, red is FP, green is FN.
Test preds:
image_batch = np.stack([val_augs(image=img)['image'] for img in test_images_medseg], axis=0)
test_preds = model.predict_on_batch(image_batch)
test_masks_prediction = test_preds > 0.5
visualize(image_batch, test_masks_prediction, num_samples=len(test_images_medseg))
Resize prediction to original size:
import scipy
test_masks_prediction_original_size = scipy.ndimage.zoom(test_masks_prediction[..., :-2], (1, 2, 2, 1), order=0)
test_masks_prediction_original_size.shape
import pandas as pd

pd.DataFrame(
             data=np.stack((np.arange(len(test_masks_prediction_original_size.ravel())), 
                            test_masks_prediction_original_size.ravel().astype(int)),
                            axis=-1), 
             columns=['Id', 'Predicted'])\
.set_index('Id').to_csv('submission.csv')